In [1]:
from typing import Dict, List

        
class Variable:
    def __init__(self, indicative=None, code=None, value=0):
        if code is not None:
            self.indicative = Variable.decode_variable(code)
        else:
            self.indicative = indicative
        self.value = value

    def do_nothing(self):
        self.value = self.value
        
    def add_one(self):
        self.value += 1

    def sub_one(self):
        if self.value != 0:
            self.value -= 1

    def is_not_zero(self):
        return self.value != 0

    @staticmethod
    def decode_variable(code: int) -> str:
        index = code//2
        
        if code==1:
            return 'Y'
        elif code%2 == 0:
            return 'X' + str(index)
        else:
            return 'Z' + str(index)
            
    def __repr__(self):
        return self.indicative


class Label:
    def __init__(self, indicative=None, code=None):
        if code is not None:
            self.indicative = Label.decode_label(code)
        else:
            self.indicative = indicative

    @staticmethod
    def decode_label(code: int) -> str:
        index = (code+4)//5
        characters = ['A', 'B', 'C', 'D', 'E']
        
        return characters[(code-1)%5] + str(index)
        
    def __repr__(self):
        return self.indicative
        
class Instruction:
    def __init__(self, code=None):
        if code is not None:
            self.label_indicative, self.abc_code  = Instruction.decode_instruction(code)
            
    @staticmethod
    def __split_number(code: int):
        num = code + 1
        
        two_factor = 0
        while num%2 == 0:
            num /= 2
            two_factor += 1
            
        x = int(two_factor)
        y = int((num-1)//2)

        return (x,y)

    @staticmethod
    def decode_instruction(code: int) -> str:
        a, t = Instruction.__split_number(code)
        b, c = Instruction.__split_number(t)

        label_indicative = None
        if a != 0:
            label_indicative = str(Label(code=a))
            
        return label_indicative, (a,b,c)


class Memory:
    INSTRUCTIONS_LIST = []
    VARIABLES_SET = {}
    EXIT_INDEX = 0
    LABEL_TO_INDEX_SET = {}
            
    @staticmethod
    def add_variable(indicative: str, variable):
        Memory.VARIABLES_SET[indicative] = variable

    @staticmethod
    def feed_variables(variables_values: List):
        Memory.VARIABLES_SET = {}
        for i in range(1, len(variables_values) + 1):
            variable = Variable(indicative=f'X{i}', value=variables_values[i-1])
            Memory.add_variable(f'X{i}', variable)
    
    @staticmethod
    def find_variable(indicative: str):
        if indicative in Memory.VARIABLES_SET:
            return Memory.VARIABLES_SET[indicative]
        else:
            Memory.VARIABLES_SET[indicative] = Variable(indicative=indicative, value=0)
            return Memory.VARIABLES_SET[indicative]
        
    @staticmethod
    def feed_instructions(instruction_codes: List):
        instructions = [Instruction(code=instruction_code) for instruction_code in instruction_codes]
        Memory.INSTRUCTIONS_LIST = instructions + [None]
        Memory.LABEL_TO_INDEX_SET = {'E' : len(Memory.INSTRUCTIONS_LIST) + 1}
        Memory.EXIT_INDEX = Memory.LABEL_TO_INDEX_SET['E']
        
        for instruction_index in range(1, len(instructions)+1):
            label_indicative = instructions[instruction_index-1].label_indicative
            if label_indicative is not None:
                Memory.LABEL_TO_INDEX_SET.update({label_indicative: instruction_index})

    @staticmethod
    def find_instruction_index_by_label(label_indicative: str):
        return Memory.LABEL_TO_INDEX_SET.get(label_indicative, Memory.EXIT_INDEX)

    @staticmethod
    def find_instruction_by_index(index: int):
        return Memory.INSTRUCTIONS_LIST[index-1]

    @staticmethod
    def get_snapshot():
        return {indicative: Memory.VARIABLES_SET[indicative].value for indicative in Memory.VARIABLES_SET}
        

class Program:
    def __init__(self, instruction_codes: List, input_variables: List):
        Memory.feed_instructions(instruction_codes)
        Memory.feed_variables(input_variables)

    @staticmethod
    def run_instruction(instruction: Instruction):
        if instruction is None:
            return -1  # app is finished
            
        a, b, c = instruction.abc_code
        instruction_type = b
        V_indicative = str(Variable(code=c+1))
        V = Memory.find_variable(V_indicative)
        
        if instruction_type == 0:
            V.do_nothing()  # V <- V
            return 0  # success
            
        elif instruction_type == 1:
            V.add_one() # V <- V+1
            return 0  # success
            
        elif instruction_type == 2:
            V.sub_one() # V <- V-1
            return 0  # success
            
        else:
            goto_label_code = instruction_type-2
            L_p = str(Label(code=goto_label_code))

            if V.is_not_zero():  # IF V != 0 GOTO L_p
                return Memory.find_instruction_index_by_label(L_p)  # next line index
                
            return 0  # success
            
    def run(self):
        snapshots = []
        line_number = 1
        while line_number != -1:  # -1 status means that the running is over (by going to E label or reaching the end of code)
            snapshots.append((line_number, Memory.get_snapshot()))
            
            instruction = Memory.find_instruction_by_index(line_number)
            run_status = Program.run_instruction(instruction)

            if run_status == 0:
                line_number += 1
            else:
                line_number = run_status
                
        return snapshots


def print_snapshots(snapshots):
    x_indices = set()
    z_indices = set()
    for snapshot in snapshots:
        for var in snapshot[1]:
            if var[0]=='X':
                x_indices.add(int(var[1:]))
            if var[0]=='Z':
                z_indices.add(int(var[1:]))
    max_x, max_z = max(x_indices), max(z_indices)
    
    for i in range(len(snapshots)-1):
        snapshot_line, snapshot_vars = snapshots[i]
        
        print(snapshot_line, end=' ')
        
        for x_indice in range(1, max_x+1):
            print(snapshot_vars.get(f'X{x_indice}', 0), end=' ')

        for z_indice in range(1, max_z+1):
            print(snapshot_vars.get(f'Z{z_indice}', 0), end=' ')

        print(snapshot_vars.get(f'Y', 0), end='\n') 

In [2]:
user_input = '45 34 350 2 46 \n 2 1' # input()
instruction_codes, input_variables = user_input.split('\n')

instruction_codes = instruction_codes.strip().split()
instruction_codes = [int(i) for i in instruction_codes]

input_variables = input_variables.strip().split()
input_variables = [int(i) for i in input_variables]

p = Program(instruction_codes, input_variables)
snapshots = p.run()

print_snapshots(snapshots)

1 2 1 0 0 0 0
2 1 1 0 0 0 0
3 1 1 0 0 1 0
4 1 1 0 0 1 0
5 1 1 0 0 1 1
1 1 1 0 0 1 1
2 0 1 0 0 1 1
3 0 1 0 0 2 1
4 0 1 0 0 2 1
5 0 1 0 0 2 2
